In [49]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
import sys
import os
sys.path.append(os.path.abspath(".."))  # Add project root to Python path
from src.optimiser import EquityOptimiser
from download_data import load_optimization_data

## Load Data

In [52]:
# Load the data
data = load_optimization_data(Path(os.path.abspath(".") + "/spx_data"), 500)
tickers = data['tickers']
expected_returns = data['expected_returns']
covariance_matrix = data['covariance_matrix']
adv = data['adv']

## Investor 1 - Loves Risk

### Let's go YOLO!

In [53]:
optimiser = EquityOptimiser(
    expected_returns=expected_returns,
    covariance_matrix=covariance_matrix
)

# Run optimization
weights, port_return, port_risk = optimiser.optimise_return_given_risk(99)

# Create results DataFrame
results_df = pd.DataFrame({
    'Ticker': tickers,
    'Weight': weights,
    'Mean': expected_returns,
    'Variance': np.diag(covariance_matrix),
    'ADV': adv,
    'Abs Weights': np.abs(weights)
}).sort_values('Abs Weights', ascending=False)

/Users/chakshu/Desktop/sherwood/equity-optimiser/.venv/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:245: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)


In [54]:
print("Optimal Portfolio:")
print(results_df.head(5))  # Show top holdings
print(f"Portfolio Return: {port_return:.2%}")
print(f"Portfolio Risk: {port_risk}")

Optimal Portfolio:
    Ticker     Weight      Mean  Variance           ADV  Abs Weights
163    LNT -16.443801  0.084540  0.044385  8.060670e+07    16.443801
462    PNR  15.498584  0.235138  0.083089  9.468150e+07    15.498584
80      FI  15.477927  0.339374  0.055535  3.912301e+08    15.477927
422     WM -15.180482  0.195182  0.036192  2.861186e+08    15.180482
294    CMS  14.782473  0.090047  0.040503  1.270666e+08    14.782473
Portfolio Return: 7383.42%
Portfolio Risk: 6.068928207345589


### Those were amazing returns, but crazy risk and weights!!

### Maybe let's go long only

In [55]:
optimiser.set_weights_bound(0,1)
weights, port_return, port_risk = optimiser.optimise_return_given_risk(99)
# Create results DataFrame
results_df = pd.DataFrame({
    'Ticker': tickers,
    'Weight': weights,
    'Mean': expected_returns,
    'Variance': np.diag(covariance_matrix),
    'ADV': adv,
    'Abs Weights': np.abs(weights)
}).sort_values('Abs Weights', ascending=False)
print("Optimal Portfolio:")
print(results_df.head(5))  # Show top holdings
print(f"Portfolio Return: {port_return:.2%}")
print(f"Portfolio Risk: {port_risk}")

/Users/chakshu/Desktop/sherwood/equity-optimiser/.venv/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:245: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)


Optimal Portfolio:
    Ticker        Weight      Mean  Variance           ADV   Abs Weights
412   SMCI  3.478677e-01  1.340217  0.861255  2.092584e+09  3.478677e-01
480    VST  3.144219e-01  0.834528  0.220911  3.360270e+08  3.144219e-01
172   PLTR  3.003052e-01  1.033747  0.487787  1.474923e+09  3.003052e-01
94     CEG  3.740524e-02  0.792791  0.225693  3.732599e+08  3.740524e-02
13     TPL  1.486276e-10  0.599806  0.202519  9.011558e+07  1.486276e-10
Portfolio Return: 106.87%
Portfolio Risk: 0.4972155106364174


### Ah slightly less but still good returns, a bit better with risk
### But Top-3 rules 

### Maybe limit top concentration

In [56]:
optimiser.set_top_k_limit(5, 0.6)

In [57]:
weights, port_return, port_risk = optimiser.optimise_return_given_risk(99)
# Create results DataFrame
results_df = pd.DataFrame({
    'Ticker': tickers,
    'Weight': weights,
    'Mean': expected_returns,
    'Variance': np.diag(covariance_matrix),
    'ADV': adv,
    'Abs Weights': np.abs(weights)
}).sort_values('Abs Weights', ascending=False)
print("Optimal Portfolio:")
print(results_df.head(5))  # Show top holdings
print(f"Portfolio Return: {port_return:.2%}")
print(f"Portfolio Risk: {port_risk}")

/Users/chakshu/Desktop/sherwood/equity-optimiser/.venv/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:245: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)


Optimal Portfolio:
    Ticker    Weight      Mean  Variance           ADV  Abs Weights
412   SMCI  0.286024  1.340217  0.861255  2.092584e+09     0.286024
172   PLTR  0.146626  1.033747  0.487787  1.474923e+09     0.146626
480    VST  0.055783  0.834528  0.220911  3.360270e+08     0.055783
94     CEG  0.055783  0.792791  0.225693  3.732599e+08     0.055783
13     TPL  0.055783  0.599806  0.202519  9.011558e+07     0.055783
Portfolio Return: 88.39%
Portfolio Risk: 0.3977762113058913


### Lower return and risk, but better with concentration